In [1]:
import pandas as pd
import numpy as np
from functions import *
import re
pd.set_option('future.no_silent_downcasting', True)

%load_ext autoreload
%autoreload 2


**Table of contents**<a id='toc0_'></a>    
- [Clean trade data](#toc1_1_)    
  - [Extract valid stocks and informtion on them](#toc1_2_)    
    - [Clean P/E, Turnover, Bid, and Ask data](#toc1_2_1_)    
    - [P/E](#toc1_2_2_)    
    - [Turnover](#toc1_2_3_)    
    - [Ask](#toc1_2_4_)    
    - [Bid](#toc1_2_5_)    
    - [divdend](#toc1_2_6_)    
    - [Join all dataframes](#toc1_2_7_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [2]:
# set paths to data
path = '/Users/johan/Library/CloudStorage/GoogleDrive-johan.oelgaard@gmail.com/My Drive/04 Økonomi/10 Thesis/Data'

# read monthly market data from eikon
monthly = 'eikon_monthly.xlsx'
eikon_dfs = pd.read_excel(path + '/' + monthly, sheet_name=None)
eikon_keys = eikon_dfs.keys()

## <a id='toc1_1_'></a>[Clean trade data](#toc0_)

In [3]:
# clean trade data
trade_values_df = eikon_dfs['Trade Values'].iloc[:,1:]

# set up multi-index for the columns
trade_values_df.columns = pd.MultiIndex.from_arrays(trade_values_df.iloc[:2].values)

# drop the first two rows as they are now headers
trade_values_df = trade_values_df.iloc[2:].reset_index(drop=True)

# set the first column as index
trade_values_df.set_index(trade_values_df.columns[0], inplace=True)
trade_values_df.index.name = "timestamp"
trade_values_df = trade_values_df.sort_index(axis=1, level=0)

/opt/anaconda3/lib/python3.11/site-packages/pandas/core/indexes/base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


In [4]:
# required columns:
required_columns = {"Trade Close", "Trade High", "Trade Low", "Trade Open", "Trade Volume"}

# extract all tickers from the first level of the columns
tickers = trade_values_df.columns.levels[0]

valid_tickers = []

for ticker in tickers:
    # the sub-columns (second-level) for this particular ticker
    subcols = set(trade_values_df[ticker].columns)
    
    # check if all required columns are present
    if required_columns.issubset(subcols):
        
        # now check how many valid rows the ticker has.
        subdf = trade_values_df[ticker][list(required_columns)]
        
        # count rows that are non-null in *all* required columns:
        non_null_rows = subdf.dropna(how="any").shape[0]
        
        if non_null_rows >= 13: # we uses 12 month momentum hence need at least 13 months of data to get even one valid data point
            valid_tickers.append(ticker)

# filter the original df to keep only valid tickers and all their second-level columns:
trade_df = trade_values_df.loc[:, (valid_tickers, slice(None))]

## <a id='toc1_2_'></a>[Extract valid stocks and informtion on them](#toc0_)

In [5]:
# clean stock names
stocks_df = eikon_dfs['Unique Stocks'].iloc[:,0:3]
# rename Code to Ticker
stocks_df.rename(columns={'Code': 'Ticker'}, inplace=True)

# use valid_tickers to filter the stocks_df
stocks_df = stocks_df[stocks_df['Ticker'].isin(valid_tickers)].reset_index(drop=True)

/var/folders/yw/jsw5n53s1cb1s2q6tt0msrm00000gn/T/ipykernel_58064/1566292336.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks_df.rename(columns={'Code': 'Ticker'}, inplace=True)


In [6]:
nace_df = eikon_dfs['NACE'].iloc[1:,1:3]

# rename columns
nace_df.columns = ['Ticker', 'NACE']

# identify the NACE codes
nace_df['NACE'] = nace_df['NACE'].str.extract(r'\((\d+(?:\.\d+)?)\)$')

# manually map remaining NACE codes to companies
manual_nace = {'CEMAT.CO':'68.20',
               'CICC.CO^L01':'70.10',
               'DAI.CO^A02':'70.10',
               'GR4.CO^A05':'80.10',
               'GR4n1.CO^J04':'80.10',
               'GR4n2.CO^J04':'80.10',
               'IFAC.CO^D03':'64.30',
               'INVb.CO^F05':'64.30',
               'IPFCa.CO^G02':'70.10',
               'IPFCb.CO^G02':'70.10',
               'OBJCa.CO^D02':'62.01',
               'OBJCb.CO^D02':'62.01',
               'ORSTED.CO':'35.11',
               'POFLSb.CO^H06':'64.30',
               'POKAP.CO^B06':'64.30',
               'RADIb.CO^C04':'32.50',
               'TRMC.CO^H02':'64.19',
               'VEND.CO^C02':'64.19'}

for ticker, nace_code in manual_nace.items():
    if ticker in nace_df['Ticker'].values:
        nace_df.loc[nace_df['Ticker'] == ticker, 'NACE'] = nace_code
    else:   
        print(f"Ticker {ticker} not found in NACE DataFrame.")

# split the NACE codes into separate columns
nace_df['NACE'] = nace_df['NACE'].str.split('.', expand=True)[0]

In [7]:
shares_df = eikon_dfs['Outstanding Shares'].iloc[:,1:]

# make first row the header
shares_df.columns = shares_df.iloc[0]
shares_df = shares_df[1:]

# rename the first column to 'Ticker'
shares_df.rename(columns={shares_df.columns[0]: 'Ticker'}, inplace=True)

# set columns to type numeric and findf the valid first occurrence
shares_df.iloc[:, 1:] = shares_df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
shares_df['Shares'] = shares_df.apply(lambda row: first_valid(row, shares_df.columns[1:]),axis=1)

# drop all columns except 'Ticker' and 'Shares'
shares_df = shares_df[['Ticker', 'Shares']]

# manually add shares for some tickers
manual_shares = {
    'ALBCb.CO^F02': 577000,
    'DAI.CO^A02': 291250,
    'FRINV.CO^A02': 803451,
    'IFAC.CO^D03': 450000,
    'IPFCa.CO^G02': 4463748,
    'IPFCb.CO^G02': 4463748,
    'SAMC.CO^G03': 205190,
    'TRMC.CO^H02': 180000,
    'VEND.CO^C02': 155000
}

for ticker, shares in manual_shares.items():
    if ticker in shares_df['Ticker'].values:
        shares_df.loc[shares_df['Ticker'] == ticker, 'Shares'] = shares
    else:   
        print(f"Ticker {ticker} not found in Shares DataFrame.")

In [8]:
# join the stocks_df with the nace_df df
stocks_df = stocks_df.merge(shares_df, how='left', on='Ticker')
stocks_df = stocks_df.merge(nace_df, how='left', on='Ticker')

# rename columns
stocks_df.rename(columns={
    'Ticker': 'ticker',
    'Name': 'name',
    'Shares': 'shares',
    'Code incl. Expiration':'code_incl_expiration',
    'NACE': 'NACE',
}, inplace=True)

# save as stocks
stocks_df.to_csv('data/stocks.csv', index=False)

### <a id='toc1_2_1_'></a>[Clean P/E, Turnover, Bid, and Ask data](#toc0_)

### <a id='toc1_2_2_'></a>[P/E](#toc0_)

In [9]:
pe_ratio_df = eikon_dfs['PE Ratio'].iloc[:,1:]

# convert the extracted values to strings to prevent dtype inference issues
pe_ratio_df.columns = pd.MultiIndex.from_arrays(pe_ratio_df.iloc[:2].values)

# drop the first two rows as they are now headers
pe_ratio_df = pe_ratio_df.iloc[2:].reset_index(drop=True)

# set the first column as index
pe_ratio_df.set_index(pe_ratio_df.columns[0], inplace=True)
pe_ratio_df.index.name = "timestamp"

# filter to only include valid tickers
pe_ratio_df = pe_ratio_df.loc[:, (valid_tickers, slice(None))]

# rename all the columns called 'PERATIO' to 'PE Ratio'
pe_ratio_df.columns = [(ticker, 'PE Ratio') if col == 'PERATIO' else (ticker, col) for ticker, col in pe_ratio_df.columns]

# Count columns (tickers) where all values are NaN
count_no_pe = pe_ratio_df.isna().all(axis=0).sum()

print(f"Companies with no PE ratio in the entire period: {count_no_pe}")

Companies with no PE ratio in the entire period: 69


/opt/anaconda3/lib/python3.11/site-packages/pandas/core/indexes/base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


### <a id='toc1_2_3_'></a>[Turnover](#toc0_)

In [10]:
turnover_df = eikon_dfs['Turnover'].iloc[:,1:]

# convert the extracted values to strings to prevent dtype inference issues
turnover_df.columns = pd.MultiIndex.from_arrays(turnover_df.iloc[:2].values)

# drop the first two rows as they are now headers
turnover_df = turnover_df.iloc[2:].reset_index(drop=True)

# set the first column as index
turnover_df.set_index(turnover_df.columns[0], inplace=True)
turnover_df.index.name = "timestamp"

# filter to only include valid tickers
turnover_df = turnover_df.loc[:, (valid_tickers, slice(None))]

/opt/anaconda3/lib/python3.11/site-packages/pandas/core/indexes/base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


### <a id='toc1_2_4_'></a>[Ask](#toc0_)

In [11]:
ask_df = eikon_dfs['Ask'].iloc[:,1:]

# convert the extracted values to strings to prevent dtype inference issues
ask_df.columns = pd.MultiIndex.from_arrays(ask_df.iloc[:2].values)

# drop the first two rows as they are now headers
ask_df = ask_df.iloc[2:].reset_index(drop=True)

# set the first column as index
ask_df.set_index(ask_df.columns[0], inplace=True)
ask_df.index.name = "timestamp"
# filter to only include valid tickers
ask_df = ask_df.loc[:, (valid_tickers, slice(None))]

/opt/anaconda3/lib/python3.11/site-packages/pandas/core/indexes/base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


### <a id='toc1_2_5_'></a>[Bid](#toc0_)

In [12]:
bid_df = eikon_dfs['Bid'].iloc[:,1:]

# convert the extracted values to strings to prevent dtype inference issues
bid_df.columns = pd.MultiIndex.from_arrays(bid_df.iloc[:2].values)

# drop the first two rows as they are now headers
bid_df = bid_df.iloc[2:].reset_index(drop=True)

# set the first column as index
bid_df.set_index(bid_df.columns[0], inplace=True)
bid_df.index.name = "timestamp"

# filter to only include valid tickers
bid_df = bid_df.loc[:, (valid_tickers, slice(None))]

/opt/anaconda3/lib/python3.11/site-packages/pandas/core/indexes/base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


### <a id='toc1_2_6_'></a>[divdend](#toc0_)

In [13]:
div_df = eikon_dfs['Dividend'].iloc[:,1:]

# set the first row as header
div_df.columns = div_df.iloc[0]
div_df = div_df[1:]

# set first name of first column as ticker
div_df.rename(columns={div_df.columns[0]: 'ticker'}, inplace=True)

# if divdend pay date is not available use date, if adj net divdend is not available use adj gross divdend
div_df['div'] = div_df.apply(lambda row: first_valid(row, ['Adjusted Net Dividend Amount', 'Adjusted Gross Dividend Amount']), axis=1)
div_df['timestamp'] = div_df.apply(lambda row: first_valid(row, ['Dividend Pay Date', 'Date']), axis=1)

# drop all columns except 'Ticker' and 'divdend'
div_df = div_df[['ticker', 'timestamp', 'div']]

# convert timestamp to datetime
div_df['timestamp'] = pd.to_datetime(div_df['timestamp'], format='%Y-%m-%d', errors='coerce')

# convert div to numeric
div_df['div'] = pd.to_numeric(div_df['div'], errors='coerce')

# # create a column with yearly divdend
# div_df['dyearly'] = div_df.groupby('ticker')['div'].transform(lambda x: x.fillna(0).rolling(12).sum())

# convert timestamp to end of month
div_df['timestamp'] = div_df['timestamp'] + pd.offsets.MonthEnd(0)

#sum the divdends for each ticker and month
div_df = div_df.groupby(['ticker', 'timestamp'], as_index=False).agg({'div': 'sum'})

div_wide = div_df.pivot(index='timestamp', columns='ticker', values='div')

# add a second level under each ticker called "divdend"
div_wide.columns = pd.MultiIndex.from_product(
    [div_wide.columns, ['div']],
    names=['ticker', 'metric']
)

# add missing tickers from valid_tickers
for ticker in valid_tickers:
    if ticker not in div_wide.columns.get_level_values(0):
        div_wide[ticker, 'div'] = np.nan

# filter to isin valid tickers
div_wide = div_wide.loc[:, (valid_tickers, slice(None))]

# set nan to 0
div_wide = div_wide.fillna(0)

# sorted datetime index
div_wide.index = pd.to_datetime(div_wide.index)
div_wide = div_wide.sort_index()

# slice out just the divdend columns: a DF of shape (time × tickers)
divs = div_wide.xs('div', level='metric', axis=1)

# compute a 365‑day rolling sum on each column independently
yearly = divs.rolling(window='365D').sum()

# re‑label its columns to match MultiIndex 
yearly.columns = pd.MultiIndex.from_product(
    [yearly.columns, ['div_annual']],
    names=div_wide.columns.names
)
div_wide = pd.concat([div_wide, yearly], axis=1)


# compute “initiated” and “halted” on the yearly divdend
year = div_wide.xs('div_annual', level='metric', axis=1)
prev_year = year.shift(1)

# initiated: was ≤0 or NaN, now >0
initiated = ((prev_year.fillna(0) == 0) & (year > 0)).astype(int)

# halted: was >0, now ≤0 or NaN
halted = ((prev_year > 0) & (year.fillna(0) == 0)).astype(int)

# label those flags as their own metrics
initiated.columns = pd.MultiIndex.from_product(
    [initiated.columns, ['divi']],
    names=div_wide.columns.names
)
halted.columns = pd.MultiIndex.from_product(
    [halted.columns, ['divo']],
    names=div_wide.columns.names
)

# 8) final concat and sort
div_wide = pd.concat([div_wide, initiated, halted], axis=1)
div_wide = div_wide.sort_index(axis=1, level=0)

### <a id='toc1_2_7_'></a>[Join all dataframes](#toc0_)

In [14]:
df = trade_df.join([
    # pe_ratio_df, 
    turnover_df, 
    ask_df, 
    bid_df,
    ],how='outer')

df = df.join(div_wide, how='left')

# sort columns by the first level of the multi-index
df = df.sort_index(axis=1, level=0)

# display(df)

In [15]:
# create an IndexSlice for easier multi-index slicing
idx = pd.IndexSlice
# loop over the tickers that are actually in the df
for ticker in df.columns.get_level_values(0).unique():
    # extract the sub-dataframe for this ticker using .loc with IndexSlice
    subdf = df.loc[:, idx[ticker, :]]
    # find the index range where the ticker has any valid data
    valid_idx = subdf.dropna(how='all').index
    # use backward fill in the date range
    df.loc[valid_idx.max():valid_idx.min(), idx[ticker, :]] = df.loc[valid_idx.max():valid_idx.min(), idx[ticker, :]].bfill()

df = df.rename(columns={'Ask Close':'ask',
                        'Bid Close':'bid',
                        'PE Ratio':'pe_ratio',
                        'Trade Close':'adjclose',
                        'Trade High':'high',
                        'Trade Low':'low',
                        'Trade Open':'open',
                        'Trade Volume':'volume',
                        'Turnover':'turnover'})

# save df
df.to_csv('data/trade.csv', index=True)